In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('/Users/hzm/Desktop/YangTeng/code/venv/code/00.module')
import Proxy
import UA
import time

# = = = = = = = = = = = = = = = = = =

import pandas as pd
import random
import requests
import json

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu.xlsx',
                        header=0,
                        dtype=str)
list_item = df_menu['Item_Number'].to_list()

print('总数量：' + str(len(list_item)))
print()

df = pd.DataFrame()

# = = = = = = = = = = = = = = = = = =

work = Queue()
for item in list_item:
    work.put_nowait(item)

# = = = = = = = = = = = = = = = = = =

def crawler():
    global df
    
    while not work.empty():
        item = work.get_nowait()
        
        # = = = = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                list_appid = ['tfy6d12dc-19a5-4093-9f00-f276d950f77',
                              'picheng11-5163-4258-be21-a5629527a07',
                              'Shanghai-b2aa-4eed-82c3-b1e42e2e9179',
                              'danjidin-f8b1-4e84-adce-026944025ceb',
                              'ibay365a-b93e-46e7-8b46-6a33b716ab2a',
                              '-IBAY365-PRD-08e35c535-85b731b3']
                
                index_appid = random.randint(0, 5)

                developer = 'http://106.75.138.2/index.php/api/GetAppToken?auth=757F054AA22E1711B598A89EB276EDC193933EC8&appid=' + list_appid[index_appid]
                
                resp = requests.get(developer).text
                token = json.loads(resp)['getapptokenResponse']['token']

                # = = = = = = = = = = = = = = = = = =

                headers = {'X-EBAY-API-IAF-TOKEN': token}
                resp = requests.get('https://open.api.ebay.com/shopping?callname=GetSingleItem&appid=' + list_appid[index_appid] + '&responseencoding=JSON&siteid=0&version=967&IncludeSelector=Details,Compatibility,ItemSpecifics&ItemID=' + item, headers=headers, proxies=Proxy.get_Proxy_requests()).text

                info = json.loads(resp)
                if info['Ack'] != 'Success':
                    continue
                
                # = = = = = = = = = = = = = = = = = =
                
                # Item.Title - None
                title = info['Item']['Title']
                
                # = = = = = = = = = = = = = = = = = =
                
                # Item.CurrentPrice - None
                price = info['Item']['CurrentPrice']['CurrencyID'] + ' ' + str(info['Item']['CurrentPrice']['Value'])
                
                # = = = = = = = = = = = = = = = = = =
                
                # Item.QuantitySold - Details
                sold = info['Item']['QuantitySold']
                
                # = = = = = = = = = = = = = = = = = =
                
                # Item.ItemCompatibilityList - Compatibility
                if 'ItemCompatibilityList' in info['Item']:
                    vehicle = info['Item']['ItemCompatibilityList']['Compatibility']
                else:
                    vehicle = ''
                
                # = = = = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                for list_name_value in vehicle:
                    make = ''
                    model = ''
                    year = ''
                    for name_value in list_name_value['NameValueList']:
                        if 'Name' in name_value and 'Make' in name_value['Name']:
                            make = name_value['Value'][0]
                        elif 'Name' in name_value and 'Model' in name_value['Name']:
                            model = name_value['Value'][0]
                        elif 'Name' in name_value and 'Year' in name_value['Name']:
                            year = name_value['Value'][0]

                    if make not in dict_vehicle:
                        dict_vehicle[make] = {model: [year]}
                    else:
                        if model not in dict_vehicle[make]:
                            dict_vehicle[make][model] = [year]
                        else:
                            dict_vehicle[make][model].append(year)

                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        list_vehicle.append(make + ' ' + model + ' ' + min(dict_vehicle[make][model]) + '-' + max(dict_vehicle[make][model]))

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = = = = =
                
                # Item.GalleryURL - None
                if 'GalleryURL' in info['Item']:
                    src = info['Item']['GalleryURL']
                elif 'PictureURL' in info['Item']:
                    if len(info['Item']['PictureURL']) == 0:
                        src = ''
                    else:
                        src = info['Item']['PictureURL'][0]
                        
                # = = = = = = = = = = = = = = = = = =
                
                # Item.ItemSpecifics - ItemSpecifics
                if 'ItemSpecifics' in info['Item']:
                    details = info['Item']['ItemSpecifics']['NameValueList']
                else:
                    details = ''
                
                # = = = = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Item_Number': item,
                                         'Title': title,
                                         'Price': price,
                                         'Sold': sold,
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Src': src}])
                
                # = = = = = = = = = = = = = = = = = =
                
                for name_value in details:
                    df_temp[name_value['Name'].lower()] = ';'.join(name_value['Value'])
                    
                status = 'ok'
                
                break

            except:
                time.sleep(3)
                continue

        # = = = = = = = = = = = = = = = = = =
        
        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Item_Number': item}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        
        print(item +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('输出中...')
df.to_excel('./developer.xlsx', index=False)
print()
print('搞定！')

总数量：5000

234562737516  <->  [ok] - 剩余数量：4990
234526368504  <->  [ok] - 剩余数量：4989
373603464917  <->  [ok] - 剩余数量：4988
231999675299  <->  [ok] - 剩余数量：4987
374042267679  <->  [ok] - 剩余数量：4986
235077386817  <->  [ok] - 剩余数量：4985
353587242099  <->  [ok] - 剩余数量：4984
235052251147  <->  [ok] - 剩余数量：4983
234041086459  <->  [ok] - 剩余数量：4982
354561339328  <->  [ok] - 剩余数量：4981
353522049196  <->  [ok] - 剩余数量：4980
352565039734  <->  [ok] - 剩余数量：4979
373652551492  <->  [ok] - 剩余数量：4978
373651425472  <->  [ok] - 剩余数量：4977
353578655655  <->  [ok] - 剩余数量：4976
373732678525  <->  [ok] - 剩余数量：4975
374784716335  <->  [ok] - 剩余数量：4974
354852938667  <->  [ok] - 剩余数量：4973
233000180565  <->  [ok] - 剩余数量：4972
234467054989  <->  [ok] - 剩余数量：4971
354883173082  <->  [ok] - 剩余数量：4970
354893391994  <->  [ok] - 剩余数量：4969
232235046609  <->  [ok] - 剩余数量：4968
234999227396  <->  [ok] - 剩余数量：4967
234054453099  <->  [ok] - 剩余数量：4966
371009356417  <->  [ok] - 剩余数量：4965
234999228354  <->  [ok] - 剩余数量：4964
372548178133  <-> 

234933360198  <->  [ok] - 剩余数量：4762
234417928151  <->  [ok] - 剩余数量：4761
373603464833  <->  [ok] - 剩余数量：4760
374678470521  <->  [ok] - 剩余数量：4759
354763673167  <->  [ok] - 剩余数量：4758
373072627303  <->  [ok] - 剩余数量：4757
354409719111  <->  [ok] - 剩余数量：4756
373652551578  <->  [ok] - 剩余数量：4755
233607980592  <->  [ok] - 剩余数量：4754
354973086228  <->  [ok] - 剩余数量：4753
353523607663  <->  [ok] - 剩余数量：4752
233297913553  <->  [ok] - 剩余数量：4751
232789575799  <->  [ok] - 剩余数量：4750
353522036950  <->  [ok] - 剩余数量：4749
353576964430  <->  [ok] - 剩余数量：4748
372560182358  <->  [ok] - 剩余数量：4747
374042263498  <->  [ok] - 剩余数量：4746
373652551444  <->  [ok] - 剩余数量：4745
373603458411  <->  [ok] - 剩余数量：4744
234041067927  <->  [ok] - 剩余数量：4743
234039582669  <->  [ok] - 剩余数量：4742
234093010074  <->  [ok] - 剩余数量：4741
233377664035  <->  [ok] - 剩余数量：4740
234526366544  <->  [ok] - 剩余数量：4739
373652565365  <->  [ok] - 剩余数量：4738
354870236926  <->  [ok] - 剩余数量：4737
234092985603  <->  [ok] - 剩余数量：4736
373658121958  <->  [ok] - 剩余

371519653234  <->  [ok] - 剩余数量：4534
371018522208  <->  [ok] - 剩余数量：4533
354718470442  <->  [ok] - 剩余数量：4532
374603862794  <->  [ok] - 剩余数量：4531
351011446136  <->  [ok] - 剩余数量：4530
234041086236  <->  [ok] - 剩余数量：4529
352824590955  <->  [ok] - 剩余数量：4528
234622295567  <->  [ok] - 剩余数量：4527
373699763913  <->  [ok] - 剩余数量：4526
372564429300  <->  [ok] - 剩余数量：4525
353578655539  <->  [ok] - 剩余数量：4524
352753749874  <->  [ok] - 剩余数量：4523
353578635129  <->  [ok] - 剩余数量：4522
351773287020  <->  [ok] - 剩余数量：4521
234478418121  <->  [ok] - 剩余数量：4520
234041084104  <->  [ok] - 剩余数量：4519
352555038743  <->  [ok] - 剩余数量：4518
353576974806  <->  [ok] - 剩余数量：4517
231999465319  <->  [ok] - 剩余数量：4516
354363839128  <->  [ok] - 剩余数量：4515
351076436200  <->  [ok] - 剩余数量：4514
373651389255  <->  [ok] - 剩余数量：4513
373652593285  <->  [ok] - 剩余数量：4512
374678446045  <->  [ok] - 剩余数量：4511
234092963249  <->  [ok] - 剩余数量：4510
371672045039  <->  [ok] - 剩余数量：4509
353523633691  <->  [ok] - 剩余数量：4508
370825983264  <->  [ok] - 剩余

374483270735  <->  [ok] - 剩余数量：4306
354763656041  <->  [ok] - 剩余数量：4305
235059284289  <->  [ok] - 剩余数量：4304
234562738411  <->  [ok] - 剩余数量：4303
373651441074  <->  [ok] - 剩余数量：4302
374244101384  <->  [ok] - 剩余数量：4301
234478418042  <->  [ok] - 剩余数量：4300
233314563499  <->  [ok] - 剩余数量：4299
234091476893  <->  [ok] - 剩余数量：4298
352565034287  <->  [ok] - 剩余数量：4297
351589302640  <->  [ok] - 剩余数量：4296
351773533751  <->  [ok] - 剩余数量：4295
354858850319  <->  [ok] - 剩余数量：4294
353578660343  <->  [ok] - 剩余数量：4293
354546225630  <->  [ok] - 剩余数量：4292
354028972250  <->  [ok] - 剩余数量：4291
354810019281  <->  [ok] - 剩余数量：4290
374893895264  <->  [ok] - 剩余数量：4289
353578634957  <->  [ok] - 剩余数量：4288
354763668805  <->  [ok] - 剩余数量：4287
233330673850  <->  [ok] - 剩余数量：4286
374638121714  <->  [ok] - 剩余数量：4285
354763671829  <->  [ok] - 剩余数量：4284
374688164027  <->  [ok] - 剩余数量：4283
352564984513  <->  [ok] - 剩余数量：4282
233088254168  <->  [ok] - 剩余数量：4281
234830539379  <->  [ok] - 剩余数量：4280
354671010919  <->  [ok] - 剩余

352363996417  <->  [ok] - 剩余数量：4078
374209100970  <->  [ok] - 剩余数量：4077
373652545960  <->  [ok] - 剩余数量：4076
233178894041  <->  [ok] - 剩余数量：4075
373652558322  <->  [ok] - 剩余数量：4074
373652545726  <->  [ok] - 剩余数量：4073
374759029736  <->  [ok] - 剩余数量：4072
374688163280  <->  [ok] - 剩余数量：4071
374328237485  <->  [ok] - 剩余数量：4070
354363840968  <->  [ok] - 剩余数量：4069
234091509210  <->  [ok] - 剩余数量：4068
234039582502  <->  [ok] - 剩余数量：4067
234041078201  <->  [ok] - 剩余数量：4066
234039605250  <->  [ok] - 剩余数量：4065
353523607700  <->  [ok] - 剩余数量：4064
353537164818  <->  [ok] - 剩余数量：4063
233000169896  <->  [ok] - 剩余数量：4062
232780280087  <->  [ok] - 剩余数量：4061
374505983082  <->  [ok] - 剩余数量：4060
373604504371  <->  [ok] - 剩余数量：4059
235052255761  <->  [ok] - 剩余数量：4058
353522032518  <->  [ok] - 剩余数量：4057
352564209250  <->  [ok] - 剩余数量：4056
353522042910  <->  [ok] - 剩余数量：4055
374342919657  <->  [ok] - 剩余数量：4054
234526371895  <->  [ok] - 剩余数量：4053
234914506215  <->  [ok] - 剩余数量：4052
352511580715  <->  [ok] - 剩余

234039594895  <->  [ok] - 剩余数量：3850
353522076540  <->  [ok] - 剩余数量：3849
353523599324  <->  [ok] - 剩余数量：3848
234978540886  <->  [ok] - 剩余数量：3847
354666317054  <->  [ok] - 剩余数量：3846
351462215257  <->  [ok] - 剩余数量：3845
234093010009  <->  [ok] - 剩余数量：3844
234124245318  <->  [ok] - 剩余数量：3843
374678449430  <->  [ok] - 剩余数量：3842
353576952049  <->  [ok] - 剩余数量：3841
373988817462  <->  [ok] - 剩余数量：3840
353523542981  <->  [ok] - 剩余数量：3839
373604489913  <->  [ok] - 剩余数量：3838
351589408138  <->  [ok] - 剩余数量：3837
352553545004  <->  [ok] - 剩余数量：3836
353523599347  <->  [ok] - 剩余数量：3835
372497653306  <->  [ok] - 剩余数量：3834
234039605346  <->  [ok] - 剩余数量：3833
352689036540  <->  [ok] - 剩余数量：3832
354763669279  <->  [ok] - 剩余数量：3831
352564273020  <->  [ok] - 剩余数量：3830
354863820818  <->  [ok] - 剩余数量：3829
352377471768  <->  [ok] - 剩余数量：3828
234041067893  <->  [ok] - 剩余数量：3827
374042263844  <->  [ok] - 剩余数量：3826
234693303203  <->  [ok] - 剩余数量：3825
354179581519  <->  [ok] - 剩余数量：3824
373603464813  <->  [ok] - 剩余

353738888662  <->  [ok] - 剩余数量：3622
234039667403  <->  [ok] - 剩余数量：3621
354858851035  <->  [ok] - 剩余数量：3620
373651385498  <->  [ok] - 剩余数量：3619
373651450377  <->  [ok] - 剩余数量：3618
372497372362  <->  [ok] - 剩余数量：3617
373651425522  <->  [ok] - 剩余数量：3616
373603464761  <->  [ok] - 剩余数量：3615
373603476019  <->  [ok] - 剩余数量：3614
373651443485  <->  [ok] - 剩余数量：3613
372550292436  <->  [ok] - 剩余数量：3612
232999884828  <->  [ok] - 剩余数量：3611
351772815242  <->  [ok] - 剩余数量：3610
234039620180  <->  [ok] - 剩余数量：3609
354852919600  <->  [ok] - 剩余数量：3608
372743867148  <->  [ok] - 剩余数量：3607
372544062099  <->  [ok] - 剩余数量：3606
374763038750  <->  [ok] - 剩余数量：3605
373618982947  <->  [ok] - 剩余数量：3604
234100957890  <->  [ok] - 剩余数量：3603
234091494526  <->  [ok] - 剩余数量：3602
373603464870  <->  [ok] - 剩余数量：3601
234191568248  <->  [ok] - 剩余数量：3600
234041105308  <->  [ok] - 剩余数量：3599
353522054376  <->  [ok] - 剩余数量：3598
234092988575  <->  [ok] - 剩余数量：3597
372563436947  <->  [ok] - 剩余数量：3596
354941145543  <->  [ok] - 剩余

234041089515  <->  [ok] - 剩余数量：3394
234041067887  <->  [ok] - 剩余数量：3393
374678448882  <->  [ok] - 剩余数量：3392
354028973973  <->  [ok] - 剩余数量：3391
372321518845  <->  [ok] - 剩余数量：3390
234091468767  <->  [ok] - 剩余数量：3389
374042265790  <->  [ok] - 剩余数量：3388
373604523349  <->  [ok] - 剩余数量：3387
354192555252  <->  [ok] - 剩余数量：3386
374042266612  <->  [ok] - 剩余数量：3385
374333700802  <->  [ok] - 剩余数量：3384
354028977104  <->  [ok] - 剩余数量：3383
372413812261  <->  [ok] - 剩余数量：3382
354936045596  <->  [ok] - 剩余数量：3381
234999238169  <->  [ok] - 剩余数量：3380
354167750209  <->  [ok] - 剩余数量：3379
374688162775  <->  [ok] - 剩余数量：3378
373651415575  <->  [ok] - 剩余数量：3377
373604514143  <->  [ok] - 剩余数量：3376
233061589372  <->  [ok] - 剩余数量：3375
374678439269  <->  [ok] - 剩余数量：3374
353523599491  <->  [ok] - 剩余数量：3373
353522032473  <->  [ok] - 剩余数量：3372
235026881908  <->  [ok] - 剩余数量：3371
234537801160  <->  [ok] - 剩余数量：3370
352274982664  <->  [ok] - 剩余数量：3369
374720667425  <->  [ok] - 剩余数量：3368
234041068031  <->  [ok] - 剩余

234855487664  <->  [ok] - 剩余数量：3166
374744527709  <->  [ok] - 剩余数量：3165
354852948411  <->  [ok] - 剩余数量：3164
235026546765  <->  [ok] - 剩余数量：3163
374395886778  <->  [ok] - 剩余数量：3162
374759022115  <->  [ok] - 剩余数量：3161
235052255468  <->  [ok] - 剩余数量：3160
374838087865  <->  [ok] - 剩余数量：3159
373652565445  <->  [ok] - 剩余数量：3158
374767576260  <->  [ok] - 剩余数量：3157
234950479469  <->  [ok] - 剩余数量：3156
374664188292  <->  [ok] - 剩余数量：3155
352380899878  <->  [ok] - 剩余数量：3154
354512392156  <->  [ok] - 剩余数量：3153
353998201091  <->  [ok] - 剩余数量：3152
235104853813  <->  [ok] - 剩余数量：3151
374581896403  <->  [ok] - 剩余数量：3150
374664188296  <->  [ok] - 剩余数量：3149
374872274852  <->  [ok] - 剩余数量：3148
354944899557  <->  [ok] - 剩余数量：3147
354973087370  <->  [ok] - 剩余数量：3146
235005362256  <->  [ok] - 剩余数量：3145
374193606357  <->  [ok] - 剩余数量：3144
354512392520  <->  [ok] - 剩余数量：3143
234602379681  <->  [ok] - 剩余数量：3142
353975818650  <->  [ok] - 剩余数量：3141
374777517037  <->  [ok] - 剩余数量：3140
374818628236  <->  [ok] - 剩余